In [1]:
import numpy as np
import matplotlib.pyplot as plt

def gaussian_random_field(pk, n):
    nup = int(n/2)
    wn = np.concatenate((np.arange(0, nup), np.arange(-nup, 0)))
    kx, ky = np.meshgrid(wn, wn)
    k2d = np.sqrt(kx**2 + ky**2)
    k2d[np.where(k2d==0.0)] = 1e-10
    noise = np.fft.fft2(np.random.normal(0, 1, (n, n)))
    amplitude = pk(k2d)
    amplitude[np.where(k2d==1e-10)] = 0.0
    noise1 = np.real(np.fft.ifft2(noise * amplitude))
    return (noise1 - np.mean(noise1))/np.std(noise1)

def gen_model_state(ni, nj, Vmax, Rmw):
    x = np.zeros((ni, nj, 2))
    
    return x

def EnSRF(xb, obs_loc, obs, obserr, roi):
    x = xb.copy()
    nens, ni, nj, nv = x.shape
    nobs = obs.size
    for n in range(nobs):
        yo = obs[n]
        io = obs_loc[n, 0]
        jo = obs_loc[n, 1]
        ko = obs_loc[n, 2]
        hx = x[:, int(io), int(jo), ko]
        hxm = np.mean(hx)
        hxp = hx - hxm
        innov = yo - hxm
        varo = obserr[n]**2
        varb = np.sum(hxp**2) / (nens - 1)
        for v in range(nv):
            xm = np.mean(x[:, :, :, v], axis=0)
            xp = x[:, :, :, v] - np.tile(xm, (nens, 1, 1))
            for i in range(int(max(io-roi,0)), int(min(io+roi,ni))):
                for j in range(int(max(jo-roi,0)), int(min(jo+roi,nj))):
                    cov = np.sum(xp[:, i, j] * hxp) / (nens - 1)
                    dist = np.sqrt((i-io)**2+(j-jo)**2)
                    loc = local_GC(dist, roi)
                    gain = loc * cov / (varo + varb)
                    srf = 1.0 / (1.0 + np.sqrt(varo / (varo + varb)))
                    xm[i, j] = xm[i, j] + gain * innov
                    xp[:, i, j] = xp[:, i, j] - srf * gain * hxp
                    x[:, i, j, v] = xp[:, i, j] + xm[i, j]
    return x

def local_GC(dist, cutoff):
    loc = 1
    if cutoff>0:
        r = dist / (cutoff / 2)
        if dist<cutoff/2:
            loc = (((-0.25*r + 0.5)*r + 0.625)*r - 5.0/3.0) * r**2 + 1
        if dist>=cutoff/2 and dist<cutoff:
            loc = ((((r/12.0 - 0.5)*r + 0.625)*r + 5.0/3.0)*r - 5.0)*r + 4 - 2.0/(3.0*r)
        if dist>=cutoff:
            loc = 0
    else:
        loc = 1
    return loc

def optical_flow(x1, x2, landmask, nlevel, w):
    nens, ni, nj = x1.shape
    ##normalize field so that w can have fixed behavior
    for m in range(nens):
        xmax = np.max(x1[m, :, :]); xmin = np.min(x1[m, :, :])
        x1[m, :, :] = (x1[m, :, :] - xmin) / (xmax -xmin)
        x2[m, :, :] = (x2[m, :, :] - xmin) / (xmax -xmin)
    u = np.zeros((nens, ni, nj))
    v = np.zeros((nens, ni, nj))
    ###pyramid levels
    for lev in range(nlevel, -1, -1):
        x1w = warp(x1, -u, -v)
        x1c = coarsen(x1w, 1, lev)
        x2c = coarsen(x2, 1, lev)
        niter = 1000
        xdx = 0.5*(deriv_x(x1c) + deriv_x(x2c))
        xdy = 0.5*(deriv_y(x1c) + deriv_y(x2c))
        xdt = x2c - x1c
        ###compute incremental flow using iterative solver
        du = np.zeros(xdx.shape)
        dv = np.zeros(xdx.shape)
        for k in range(niter):
            ##boundary conditions
            du[0,:] = 0; du[-1,:] = 0; du[:,0] = 0; du[:,-1] = 0
            dv[0,:] = 0; dv[-1,:] = 0; dv[:,0] = 0; dv[:,-1] = 0
            ubar = laplacian(du) + du
            vbar = laplacian(dv) + dv
            du = ubar - xdx*(xdx*ubar + xdy*vbar + xdt)/(w + xdx**2 + xdy**2)
            dv = vbar - xdy*(xdx*ubar + xdy*vbar + xdt)/(w + xdx**2 + xdy**2)
        u += sharpen(du*2**lev, lev, 1)
        v += sharpen(dv*2**lev, lev, 1)
    return u, v

def warp(x, u, v):
    xw = x.copy()
    nens, ni, nj = x.shape
    mm, ii, jj = np.mgrid[0:nens, 0:ni, 0:nj]
    xw = interp2d(x, mm, ii+u, jj+v)
    return xw

def coarsen(x, lev1, lev2):
    if lev1 < lev2:
        for k in range(lev1, lev2):
            if x.ndim == 2:
                ni, nj = x.shape
                x1 = 0.25*(x[0:ni:2, 0:nj:2] + x[1:ni:2, 0:nj:2] + x[0:ni:2, 1:nj:2] + x[1:ni:2, 1:nj:2])                
            if x.ndim == 3:
                nens, ni, nj = x.shape
                x1 = 0.25*(x[:, 0:ni:2, 0:nj:2] + x[:, 1:ni:2, 0:nj:2] + x[:, 0:ni:2, 1:nj:2] + x[:, 1:ni:2, 1:nj:2])
            if x.ndim == 4:
                nens, ni, nj, nv = x.shape
                x1 = 0.25*(x[:, 0:ni:2, 0:nj:2, :] + x[:, 1:ni:2, 0:nj:2, :] + x[:, 0:ni:2, 1:nj:2, :] + x[:, 1:ni:2, 1:nj:2, :])
            x = x1
    return x

def sharpen(x, lev1, lev2):
    if lev1 > lev2:
        for k in range(lev1, lev2, -1):
            if x.ndim == 2:
                ni, nj = x.shape
                x1 = np.zeros((ni*2, nj))
                x1[0:ni*2:2, :] = x
                x1[1:ni*2:2, :] = 0.5*(np.roll(x, -1, axis=0) + x)
                x2 = np.zeros((ni*2, nj*2))
                x2[:, 0:nj*2:2] = x1
                x2[:, 1:nj*2:2] = 0.5*(np.roll(x1, -1, axis=1) + x1)
            if x.ndim == 3:
                nens, ni, nj = x.shape
                x1 = np.zeros((nens, ni*2, nj))
                x1[:, 0:ni*2:2, :] = x
                x1[:, 1:ni*2:2, :] = 0.5*(np.roll(x, -1, axis=1) + x)
                x2 = np.zeros((nens, ni*2, nj*2))
                x2[:, :, 0:nj*2:2] = x1
                x2[:, :, 1:nj*2:2] = 0.5*(np.roll(x1, -1, axis=2) + x1)

            if x.ndim == 4:
                nens, ni, nj, nv = x.shape
                x1 = np.zeros((nens, ni*2, nj, nv))
                x1[:, 0:ni*2:2, :, :] = x
                x1[:, 1:ni*2:2, :, :] = 0.5*(np.roll(x, -1, axis=1) + x)
                x2 = np.zeros((nens, ni*2, nj*2, nv))
                x2[:, :, 0:nj*2:2, :] = x1
                x2[:, :, 1:nj*2:2, :] = 0.5*(np.roll(x1, -1, axis=2) + x1)
            x = x2
    return x

def get_scale(x, level, s):
    ns = len(level)
    xs = coarsen(x, 1, level[s])
    if s > 0:
        xc = coarsen(x, 1, level[s-1])
        xcs = sharpen(xc, level[s-1], level[s])
        xs -= xcs
    return xs

def interp2d(x, mm, io, jo):
    nens, ni, nj = x.shape
    io1 = np.floor(io).astype(int) % ni
    jo1 = np.floor(jo).astype(int) % nj
    io2 = np.floor(io+1).astype(int) % ni
    jo2 = np.floor(jo+1).astype(int) % nj
    di = io - np.floor(io)
    dj = jo - np.floor(jo)
    xo = (1-di)*(1-dj)*x[mm, io1, jo1] + di*(1-dj)*x[mm, io2, jo1] + (1-di)*dj*x[mm, io1, jo2] + di*dj*x[mm, io2, jo2]
    return xo

def deriv_x(f):
    return 0.5*(np.roll(f, -1, axis=1) - np.roll(f, 1, axis=1))

def deriv_y(f):
    return 0.5*(np.roll(f, -1, axis=2) - np.roll(f, 1, axis=2))

def laplacian(f):
    return (np.roll(f, -1, axis=1) + np.roll(f, 1, axis=1) + np.roll(f, -1, axis=2) + np.roll(f, 1, axis=2))/6 + (np.roll(np.roll(f, -1, axis=2), -1, axis=1) + np.roll(np.roll(f, -1, axis=2), 1, axis=1) + np.roll(np.roll(f, 1, axis=2), -1, axis=1) + np.roll(np.roll(f, 1, axis=2), 1, axis=1))/12 - f



In [4]:
##generate initial ensemble and visualize


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.